# Выбор локации для скважины

## **Описание проекта**

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных


Данные геологоразведки трёх регионов находятся в файлах: 
- /datasets/geo_data_0.csv
- /datasets/geo_data_1.csv
- /datasets/geo_data_2.csv

Содержание:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

### План работы

1. Загрузить и подготовить данные.
2. Обучить проверить модель для каждого региона:
 - 2.1. Разбийть данные на обучающую и валидационную выборки в соотношении 75:25.
 - 2.2. Обучить модель и сделать предсказания на валидационной выборке.
 - 2.3. Сохранить предсказания и правильные ответы на валидационной выборке.
 - 2.4. Напечатайть на экране средний запас предсказанного сырья и RMSE модели.
 - 2.5. Проанализировать результаты.
3. Подготовиться к расчёту прибыли:
 - 3.1. Все ключевые значения для расчётов сохранить в отдельных переменных.
 - 3.2. Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе. 
 - 3.3. Написать выводы по этапу подготовки расчёта прибыли.
4. Написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
 - 4.1. Выбрать скважины с максимальными значениями предсказаний. 
 - 4.2. Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
 - 4.3. Рассчитать прибыль для полученного объёма сырья.
5. Посчитать риски и прибыль для каждого региона:
 - 5.1. Применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 - 5.2. Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
 - 5.3. Написать выводы: предложить регион для разработки скважин и обосновать выбор.
6. Написать общий вывод по проделанной работе.

## **0. Инициализация необходимых библиотек и методов**

In [ ]:
pip install -U -q scikit-learn cufflinks phik shap seaborn optuna-integration

In [ ]:
try:
    from ydata_profiling import ProfileReport
except:
    !pip install -U -q Pillow # Для решения ошибки с профайлером
    !pip install -U -q ydata-profiling
    from ydata_profiling import ProfileReport

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
# Импорт основных библиотек
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

# Импорт графической части plotly
import plotly.express as px
from plotly.offline import plot



# Импорт машинной части
import shap
# Импорт библиотек необходимых на моделей регрессии

from warnings import simplefilter # Для игнорирования предупреждений о изменении некоторых моделей в следующих версия
from scipy import stats as st
from phik import phik_matrix
from phik.report import plot_correlation_matrix
from sklearn.model_selection import train_test_split

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# загружаем классы для подготовки данных
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer


# загружаем функцию для работы с метриками
from sklearn.metrics import root_mean_squared_error


# загружаем нужные модели
from sklearn.linear_model import LinearRegression


# Для проверки адекватности модели
from sklearn.dummy import DummyRegressor
# загружаем методы отбора лучших признаков
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

simplefilter(action='ignore', category=FutureWarning) # Включаем фильтр предупреждений

In [ ]:
# Задаём основные показатели размерности выборок и порядка разделения на выборки
RANDOM_STATE = 50
TEST_SIZE = 0.25

## **1. Загрузка и подготовка данных**

### 1.1. Загрузка и знакомство с данными

In [ ]:
geo_data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv',index_col='id')

In [ ]:
ProfileReport(geo_data_0, title="Profiling Report") # Получение основной информации о сведениях в тренировочных данных

In [ ]:
geo_data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv',index_col='id')

In [ ]:
ProfileReport(geo_data_1, title="Profiling Report") # Получение основной информации о сведениях в тренировочных данных

In [ ]:
geo_data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv',index_col='id')

In [ ]:
ProfileReport(geo_data_2, title="Profiling Report") # Получение основной информации о сведениях в тренировочных данных

### Выводы после знакомства с данными
- В данных отсутствуют пропуски;
- В таблице нет явных дубликатов;
- Некоторые признаки коррелируют между собой. Нужно выполнить проверку на наличие мультиколлинеарности;
- В geo_data_1 около 8% скважин обладают нулевым запасом;
- Во всех таблицах разных разброс значений признаков. Стоит над этим поразмыслить.

Данные требует дополнительного исследования и проверки на мультиколлинеарность.

### 1.2. Исследовательский анализ данных

In [ ]:
# Функция построения гистограммы данных с учётом влияния дополнительного признака
def draw_hist_columns(data, columns, color=''):
    for column in columns:
        if column in data.columns:
            if color != '':
                fig = px.histogram(data, x=column, color=color, title='Распределение столбца '+column + ' с оценкой влияния ' + color, histnorm='density')
            else:
                fig = px.histogram(data, x=column, title='Распределение столбца '+column)
            fig.show()

In [ ]:
# Строим boxplot для столбцов с некатегориальным распределением данных
def draw_boxplot_columns(data, columns, color=''):
    for column in columns:
        if color != '':
            fig = px.box(data, 
                     x=column, 
                     color=color,
                     title='Ящик с усами столбца '+ str(column) + 'с оценкой влияния ' + color
                    )
        else:
            fig = px.box(data, 
                     x=column,
                     title='Ящик с усами столбца '+ str(column)
                    )
        fig.show()

#### 1.2.1. Исследовательский анализ geo_data_0

In [ ]:
draw_hist_columns(geo_data_0, geo_data_0.columns) # Гистограмма распределения показателй признаков

In [ ]:
draw_boxplot_columns(geo_data_0, geo_data_0.columns) # Постояние ящиков с усами для признаков

**Выводы после анализа данных geo_data_0**

- Только f2 имеет нормальное распределение, и насыщен достаточным числом выбросов;
- Остальные признаки имеют шипообразную форму;
- Видимо, из-за влияния высокого влияния некоторых признаков product имеет шипообразную форму. Однако, если шипы объединить - он будет напоминать нормальное распределение;
- Объём запасов распределился от 0 до ~200тыс. баррелей нефти в данном регионе.

#### 1.2.2. Исследовательский анализ geo_data_1

In [ ]:
draw_hist_columns(geo_data_1, geo_data_1.columns)

In [ ]:
draw_boxplot_columns(geo_data_1, geo_data_1.columns)

**Выводы после анализа данных geo_data_1**
- Признак f2 напоминает дискретную форму распределения данных;
- Видимо, из-за высокого влияния показателя f2, product принял парно-дискретную форму распределения;
- В данном регионе запасы ниже. В диапазоне от 0 до 140.

#### 1.2.3. Исследовательский анализ geo_data_2

In [ ]:
draw_hist_columns(geo_data_2, geo_data_2.columns)

In [ ]:
draw_boxplot_columns(geo_data_2, geo_data_2.columns)

**Выводы после анализа данных geo_data_2**
- Все входные признаки имеют форму нормального распределения;
- Целевой признак напоминает форму нормального распределения с выступающими шипами;
- В данном регионе запас нефти варьируется от 0 до ~180тыс.баррелей нефти.

#### Общие выводы исследовательского анализа данных

- Все признаки имеют разную форму и диапазон значений для каждой таблицы данных;
- В каждом регионе разный запас нефти;
- Стоит провести корреляционный анализ данных, для определения точного влияния признаков и отсутствия мультиколлинеарности.

### 1.3. Корреляционный анализ данных

In [ ]:
pd.DataFrame.iteritems = pd.DataFrame.items

In [ ]:
# Функция отображения матрицы диаграмм рассеяния таблиц данных
def plot_scatter_matrix(data, i):
    fig = px.scatter_matrix(data, title='Диаграмма рассеяния признаков geo_data_' + str(i))
    fig.update_layout(width=1400,height=1400)
    fig.update_traces(marker=dict(size=4,line=dict(width=0.5,color='DarkSlateGrey')))
    fig.show()

In [ ]:
# Цикличный перебор моделей и построение диаграмм рассеяния
for i, data in enumerate([geo_data_0, geo_data_1, geo_data_2]):
    plot_scatter_matrix(data, i)

In [ ]:
# добавление константы для перехвата
def VIF_analyze(data, i):
    print('Анализ мультиколлинеарности данных geo_data_' + str(i))
    X = add_constant(data.drop('product', axis=1))
    int_columns = X.select_dtypes(include='number').columns.tolist()
    # расчет VIF для каждого предиктора
    VIFs = pd.DataFrame()
    VIFs['Variable'] = X[int_columns].columns
    VIFs["VIF"] = [variance_inflation_factor(X[int_columns].values, i) for i in range(len(X[int_columns].columns))]
    print(VIFs, '\n')

In [ ]:
# Цикличный перебор данных и оценка мультиколлинеарности
for i, data in enumerate([geo_data_0, geo_data_1, geo_data_2]):
    VIF_analyze(data, i)

**Выводы после оценки диаграмм рассеяния и проверки мультиколлинеарности:**
- В geo_data_0 интересная форму корреляции между признаками f0 и f1. Будто какой-то внешний показатель способен отделить зависимость на два бумеранга;
- Целевой признак лучшим образом коррелирует с признаком f2. С остальными же признаками - корреляция близка к значению 0;
- В geo_data_1 из-за формы признака f2 определить корреляцию крайне тяжело;
- В geo_data_2 f2 также демонстрирует своё влияния на целевой признак;
- Интересна форма рассения признаков f0, f1 в связке с целевым признаком;
- Мультиколлинеарности между признаками после проведения VIF тестирования обнаружено не было.

In [ ]:
# Строим матрицу корреляции признаков тренировочных данных
for i, data in enumerate([geo_data_0, geo_data_1, geo_data_2]):
    interval_cols = data.columns
    phik_overview = phik_matrix(data, interval_cols= interval_cols) 
    
    plot_correlation_matrix(
        phik_overview.values,
        x_labels=phik_overview.columns,
        y_labels=phik_overview.index,
        vmin=0, vmax=1, color_map='Reds',
        title=r'Таблица корреляции признаков данных geo_data_' + str(i),
        fontsize_factor=1.5,
        figsize=(12, 10)
    ) 
    plt.show()

#### Выводы корреляционного анализа

- Наибольшее общее влияния на целевой признак оказывает *f2*;
- Между признаками нет мультиколлинеарности и нет утечки данных;
- Все признакми по своему значимы для целевого признака.

Можно приступать к подготовке данных и обучению моделей.

### 1.4 Пайплайны кодирования 

In [ ]:
cols = ['f0', 'f1', 'f2'] # Вводим признаки

In [ ]:
# создаём общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [
     ('num', StandardScaler(), cols)
    ], 
    remainder='passthrough'
)

In [ ]:
# Финальный пайплайн
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('poly', PolynomialFeatures(degree=3)),
    ('models', LinearRegression() )
])

**Вывод по пайплайнам кодирования:**
- Был создан пайплайн кодирования признаков;
- Задали список перебора вариантов предподготовки данных для отбора лучших вариантов.

### Выводы после загрузки и подготовки данных

- Провели знакомство с данными. Оценили наличие пропусков, дубликатов;
- Выполнили исследовательский анализ данных. Изучили распределение данных, сделали общие выводы;
- Дополнили исследование корреляционным анализом. Оценили данные на наличие мультиколлинеарности;
- Реализовали пайплайн кодирования, для корректного обучения моделей и получения достоверных результатов.

## **2. Обучение и проверка модели**

### 2.1. Определение вариационной и обучающей выборки

In [ ]:
X_train, X_valid, y_train, y_valid = [], [], [], [] # Введение переменных для хранения выборок

In [ ]:
# Цикличное разбиение данных по выборкам
for data in [geo_data_0, geo_data_1, geo_data_2]:
    X = data.drop(columns='product')
    y = data['product']
    X_t, X_v, y_t, y_v = train_test_split(X, y, test_size= TEST_SIZE, random_state=RANDOM_STATE)
    X_train.append(X_t)
    X_valid.append(X_v)
    y_train.append(y_t)
    y_valid.append(y_v)

### 2.2. Обучение моделей и сохранение результатов предсказаний

In [ ]:
y_models = [] # Вводим список для хранения пайплайнов моделей

In [ ]:
# Вводим списки для хранения необходимых данных
models = []
y_pred_mean = []
y_valid_mean = []
rmse_models = []
dummy_rmse = []

# Цикличный перебор регионов, обучение и получение результатов моделей
for i in range(3):
    pipe_final.fit(X_train[i], y_train[i])

    models.append(pipe_final)
    
    y_models.append(pd.DataFrame(y_valid[i].copy()))

    y_pred = pipe_final.predict(X_valid[i])
    
    y_models[i].insert(1,'pred', y_pred)
    
    y_pred_mean.append(y_pred.mean())
    y_valid_mean.append(y_valid[i].mean())

    rmse_models.append(root_mean_squared_error(y_valid[i], y_pred))
    
    dummy_regr = DummyRegressor(strategy="mean") # Реализуем проверку модели на адекватность резульаттов
    dummy_regr.fit(X_train[i], y_train[i])
    y_pred_dummy = dummy_regr.predict(X_valid[i])
    
    dummy_rmse.append(root_mean_squared_error(y_valid[i], y_pred_dummy))
    

### 2.3. Оценка важности признаков

In [ ]:
for i in range(3):
    model = models[i]['models']
    preprocessor = models[i]['preprocessor']
    poly = models[i]['poly']
    X_train_t = preprocessor.transform(X_train[i])
    X_train_t = poly.transform(X_train_t)
    name_columns = poly.get_feature_names_out()
    X_train_t = pd.DataFrame(X_train_t, columns = name_columns)
    explainer = shap.LinearExplainer(model, X_train_t, feature_perturbation='interventional')
    shap_values = explainer.shap_values(X_train_t)

    plt.rcParams['axes.titlesize'] = 22
    plt.rcParams['axes.labelsize'] = 15
    plt.title('График анализа важности признаков')
    plt.ylabel('Признаки модели')

    # Построение графика важности признаков
    shap.summary_plot(shap_values, X_train_t, plot_type='dot', max_display=30, plot_size=(15,15))

** Выводы оценки важности признаков:**
- Поскольку интерпретировать признаки по наименования мы не можем, то можешь лишь отметить их показатели по важности;
- Введение дополнительных признаков оказалось полезно - важную значимость имеют признаки, полученные полиномиальным дополнением;
- Для первой итретьей моделей наиболее значим признак f2. Чем больше его значение, чем больше результат предсказания;
- Также важным оказался признак f0^2 - положительные значения сильно влияют на положительное предсказание;

Можно сделать общую отметку, что положительные значения наиболее важных признаков положительно сказываются на предсказании. Введение дополнительных признаков оправдало себя, как по значимости, так и мо качеству предсказания. Что будет отмечено далее.

### 2.4. Средний запас в регионах

In [ ]:
# Представление результатов в форме датафрейма. Индексы - номера регионов
res_models = pd.DataFrame(
    {'mean_product': y_valid_mean,
     'mean_pred_product': y_pred_mean,
     'rmse_model': rmse_models,
     'rmse_dummy': dummy_rmse
    })
res_models

### 2.5. Оценка результатов

- Наиболее точные предсказания достигнуты в регионе geo_data_1;
- В geo_data_1 целевой признак имел высочайший уровень корреляции с одним из признаков. Это может объяснить высокую результативность модели;
- Все модели справились лучше, чем модель предсказывающее среднее (DummyRegressor);
- Наибольший средний уровень запаса нефтепродуктов в 3-м регионе. Наименьший - в 1-м.

### Выводы после обучения и проверки моделей

- Разделили данные на тренировочную и вариационную выборки;
- Реализовали пайплайн для обучения моделей с учетом добавление новых признаков. Использовали полиномиальный метод введения новых признаков;
- Получили результаты предсказаний, вычислили ошибку;
- Провели адекватность работы моделей с помощью введения DummyRegressor;
- Лучше все удаётся предсказывать объём запасов в регионе geo_1;
- При этом, в данном регионе наименьшие запасы по оценке модели.

## **3. Подготовка к расчёту прибыли**

Условия задачи:
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [ ]:
all_points = 500 # Количество рассматриваемых точек
best_points = 200 # Число отбираемых для разработки точек
budget = 10_000_000_000 # Общий бюджет разработки в регионе
profit = 450_000 # Объём прибыли с одной тысячи баррелей

In [ ]:
borehole_budget = budget / best_points # Средний бюджет на разработку 1й скважины

raw_material_stock = borehole_budget / profit # Объём запасов необходимый для выхода в 0

print('Бюджет одной скважаны равен -',borehole_budget/ 1_000_000,'млн. рублей')
print(f'Необходимый объём запасов сырья для выхода в 0 - {raw_material_stock:.4} тыс. баррелей')

In [ ]:
for i in range(3):
    print(f'Средний объём запасов нефтепродуктов в регионе geo_data_{i}: {y_valid_mean[i]:.3}')
    suitable_boreholes = len(y_models[i].query('product >= @raw_material_stock')['product'])
    print(f'Количество скважин с достаточным для безубыточной разработки объёмом запаса: {suitable_boreholes}')
    print(f'В процентном соотношении число подходящих скважин соответствует - {suitable_boreholes/len(y_models[i]):.2%}', '\n')

### Выводы после подготовки данных к расчёту прибыли

- Ввели необходимые переменные для удобства работы с данными;
- Рассчитали средний бюджет выделяемый на разработку одной скважины - он составил 50 миллионов;
- Определили необходимый запас сырья для того, чтобы выйти в 0 - 111.5 тысяч баррелей нефти;
- Перспективнее всего, с точки зрения количества подходящих скважин, выглядит 3й регион, где порядка 38% скважин имеют запас превышающий уровень выхода на безубыточную добычу.

Если сравнивать среднее значение запаса в регионе с необходимым числом для безубыточной разработки - ни один регион не подходит. Именно поэтому так важна качественно настроенная модель, что сможет определить наиболее перспективные точки для разработки. В этом плане наиболее перспективны 2 региона - 2й и 3й. У 2го модель очень точно способна предсказывать объёмы, что крайне полезно, ведь данный регион не сильно богат на запасы. 3й же имеет наибольшее число скважин с превышающим необходимый минимум объёмов. Точный ответ будет дан далее, после проведения всех расчётов.

## **4. Расчёт прибыли и рисков** 

### 4.1. Реализация функций расчёта прибыли и оценки рисков

In [ ]:
# Функция расчёта прибыли в регионе для лучших точек
def regional_profit(true_values_sample, predicted_values_sample, profit, budget):
    n_best_point = predicted_values_sample.sort_values(ascending=False) # Сортируем в порядке убываня предсказания запасов
    selected_points = true_values_sample[n_best_point.index][:best_points] # Отбор лучших точек с реальными показателями
    return selected_points.sum() * profit - budget # Возвращаем расчёт прибыли

In [ ]:
# Функция бустрапирования
def bootstrap_process(true_values, predicted_values, profit, budget, name_data):
    state = np.random.RandomState(RANDOM_STATE)
    values = []
    # Циклично перебираем 1000 подвыборок     
    for i in range(1000):
        true_values_subsample = true_values.sample(n=all_points, replace=True, random_state=state) # Получаем подвыборку размерностью 500 скважин с учётом возможных повторений
        pred_values_subsample = predicted_values[true_values_subsample.index] # Отбираем 500 скажин для модели
        profit_region = regional_profit(true_values_subsample, pred_values_subsample, profit, budget) # Вычисляем прибыль в регионе для переданных скажин
        values.append(profit_region)
                
    values = pd.Series(values)
    mean_profit_region = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    print(f'Значения для региона: {name_data}')
    print('Средняя прибыль:', round(mean_profit_region / 1_000_000, 2), 'млн. рублей')
    print('Нижняя граница 95%-го доверительного интервала:', round(lower / 1_000_000,2), 'млн. рублей')
    print('Верхняя граница 95%-го доверительного интервала:', round(upper/ 1_000_000,2), 'млн. рублей')
    print(f'Риск убытка: {(values < 0).mean():.2%}','\n') # Есть небольшие сомнения в правильности записи


### 4.2. Расчёт прибыли и рисков по результатам моделей

In [ ]:
# Расчёт средней выручки и 95% доверительного интервала для всех регионов
for i in range(3):
    bootstrap_process(y_models[i]['product'],y_models[i]['pred'], profit, budget, 'geo_data_'+str(i))

### Выводы по оценке результатов

- Регион с наибольшей средней прибылью 3-й (geo_data_2);
- Доверительный интервал находится в диапазоне от 66 до 1140 миллионов рублей;
- Риск убытков в этом регионе находится на уровне 1.1%;
- Худший регион для добычи - 1й;
- Нижняя граница доверительного интервала предполагает убытки, что для нас крайне недопустимо.


Основываясь на выборе среднего уровня прибыли в регионе - **стоит остановить свой выбор на последнем**, так как в нём самый высокий показатель.
**Однако, есть дополнительная рекомендация изучить регион geo_data_1.** Средняя прибыль в этом регионе ниже, однако риск убытков ниже, а также нижняя граница доверительного интервала в 2.5 раза выше и находится на уровне 150 миллион рублей.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован

## Общий вывод по проекту

**Вывод по проделанной работе:**
- Познакомились с данными, провели исследовательский и корреляционный анализ данных. Изучили данные на мультиколлинеарность;
- Построили пайплайны кодирования. Добавили новые полиномиальные признаки;
- Разбили данные на валидационную и тренировочную выборки;
- Провели обучение моделей, оценили важность признаков и вывели результаты;
- Подготовились к расчёту прибыли в регионе. Ввели константы, реализовали нужные функции;
- Выполнили необходимые расчёты, дали характеристику регионов и подготовили вывод для заказчика.

**Небольшой вывод по моделям:**
- Лучший показатель у 2го региона, где была корреляция одного признака, аж 98%. Оно и не удивительно, один этот признак отлично предсказывает, а ещё и другие были;
- У 1 и 3 региона достаточно высокий показатель среднеквадратичного отклонения, но всё же он лучше, чем у Dummy-модели, и эту метрику удалось немного улучшить введением дополнительных признаков;
- Однако, хоть во 2м регионе и был низкий средний показатель запасов - модель очень точно предсказывает уровен нефтепродуктов, а потому не будет проблем с отбором лучших скажин и работой с ними. Это одна из причин рекомендаци 2го региона для нефтедобычи.

**Общие рекомендации заказчику:**
- Предоставить дополнительные сведения для более точной работы модели. Регионы 1 и 3 крайне перспективны по запасам, однако высокий уровень показателя метрики говорит о том, что модель не может точно предсказывать запас нефти, а потому нет возможности отбирать лучшие скважины;
- Провести анализ между 2 и 3м регионом, хоть общее среднее и расположено к 3му региону, я бы пригляделся ко 2му региону добычи, высокий доход при нижней границы доверительного интервала подтверждает этот довод;